<a href="https://colab.research.google.com/github/sundar-nallalagappan/BERT_Variants_Implementation/blob/main/Bert_variants_Multiclass_Classificationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("      ")

In [ ]:
!pip install -U "tensorflow==2.8.*"
!pip install -U "tensorflow-text==2.8.*"
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [ ]:
cols = ['Text', 'Mask', 'category']
df = pd.read_csv('train.csv', header=None, names=cols)
df

,Text,Mask,category
0,BOS Sir I created a new env with python 3.7 bu...,O O O O O O O O O O O O O O O O O O O O O O O ...,need_help
1,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
2,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
3,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
4,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link
...,...,...,...
3509,BOS it is not one to one resume but some advic...,O O O O O O O O O O O O O O O O,need_help
3510,BOS hi good noon EOS,O O O O O,Greetings
3511,BOS now what will year contain EOS,O O O O O O O,code_error
3512,BOS I have extracted it but didn&apos;t found ...,O O O O O O O O O O,need_help


In [ ]:
df['encoded_cat'] = df['category'].astype('category').cat.codes
df.drop('Mask', axis=1, inplace=True)
df

,Text,category,encoded_cat
0,BOS Sir I created a new env with python 3.7 bu...,need_help,93
1,BOS above problem how will it solve ?? EOS,need_help,93
2,BOS Hello sir is there anyone EOS,Greetings,6
3,BOS i think canvas login problem was due to br...,great,77
4,BOS Please provide me with the link to python ...,community_class_link,55
...,...,...,...
3509,BOS it is not one to one resume but some advic...,need_help,93
3510,BOS hi good noon EOS,Greetings,6
3511,BOS now what will year contain EOS,code_error,52
3512,BOS I have extracted it but didn&apos;t found ...,need_help,93


In [ ]:
df[df['encoded_cat'] == 93]

,Text,category,encoded_cat
0,BOS Sir I created a new env with python 3.7 bu...,need_help,93
1,BOS above problem how will it solve ?? EOS,need_help,93
7,BOS so ill just copy paste what I wanted to kn...,need_help,93
8,BOS sir any sujjestion to overcome this error?...,need_help,93
17,BOS ok can you check and tell me my registered...,need_help,93
...,...,...,...
3490,BOS Sir this is what I got EOS,need_help,93
3497,BOS i m not saying to copy i just want to know...,need_help,93
3509,BOS it is not one to one resume but some advic...,need_help,93
3512,BOS I have extracted it but didn&apos;t found ...,need_help,93


In [ ]:
df['category'].value_counts(ascending=False)

need_help                781
O                        609
Greetings                331
Thank_you                284
good_bye                 169
                        ... 
aws_deployment_error       1
assignment_matplotlib      1
file_not_found             1
aws_lex_smtp_error         1
assignment_by_github       1
Name: category, Length: 151, dtype: int64

In [ ]:
df.tail(4)

,Text,category,encoded_cat
3510,BOS hi good noon EOS,Greetings,6
3511,BOS now what will year contain EOS,code_error,52
3512,BOS I have extracted it but didn&apos;t found ...,need_help,93
3513,BOS file name or extension path name EOS,need_help,93


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text'],df['encoded_cat'])
print(f"{X_train.shape, y_train.shape}")
print(f"{X_test.shape, y_test.shape}")

((2635,), (2635,))
((879,), (879,))


Import the TF based BERT models

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder    = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Instantiate the functional Model

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess(text_input)
outputs           = bert_encoder(preprocessed_text)

# Neural network layers
l1 = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l2 = tf.keras.layers.Dense(151, activation='softmax', name="output")(l1)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l2])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
83/83 [==============================] - 1561s 19s/step - loss: 3.0979 - accuracy: 0.3347 - val_loss: 2.7202 - val_accuracy: 0.3993
Epoch 2/5
83/83 [==============================] - 1553s 19s/step - loss: 2.6871 - accuracy: 0.4156 - val_loss: 2.6114 - val_accuracy: 0.4152
Epoch 3/5
83/83 [==============================] - 1556s 19s/step - loss: 2.5761 - accuracy: 0.4266 - val_loss: 2.5561 - val_accuracy: 0.4300
Epoch 4/5
83/83 [==============================] - 1554s 19s/step - loss: 2.4882 - accuracy: 0.4364 - val_loss: 2.5276 - val_accuracy: 0.4448
Epoch 5/5
83/83 [==============================] - 1555s 19s/step - loss: 2.4195 - accuracy: 0.4490 - val_loss: 2.5238 - val_accuracy: 0.4221


In [ ]:
model.save('saved_model/bert_multiclass_model')

Inference

In [ ]:
test_text=['Hi! This is sairam from DLCVNLP batch.', 'Good Morning', 'sir any sujjestion to overcome this error?']
test_text

['Hi! This is sairam from DLCVNLP batch.',
 'Good Morning',
 'sir any sujjestion to overcome this error?']

In [ ]:
y_hat = model.predict(test_text)
y_hat

array([[2.21660623e-04, 4.37238701e-02, 2.87068106e-05, 8.65454378e-04,
        2.39117816e-03, 2.64017668e-04, 1.69778951e-02, 1.10750599e-02,
        2.05798745e-01, 8.73049954e-04, 2.67445401e-04, 6.47807959e-03,
        2.20202695e-04, 2.10121085e-04, 4.11853660e-04, 1.62114928e-04,
        8.34476668e-05, 1.97970774e-03, 1.63083095e-02, 1.36606805e-02,
        1.27414860e-05, 6.61676226e-04, 3.40020459e-04, 2.78122287e-04,
        1.22840877e-03, 1.56804896e-03, 1.71729140e-02, 8.21670983e-04,
        3.11507873e-04, 8.38012807e-03, 1.19921309e-03, 2.85271183e-02,
        1.12620788e-03, 9.90732588e-06, 4.51175310e-06, 2.71761470e-04,
        4.70723200e-04, 2.03552463e-05, 6.93391703e-05, 4.62208549e-03,
        3.45691363e-03, 3.46806133e-04, 1.01471287e-05, 2.89619173e-04,
        3.06081469e-03, 4.02370672e-04, 8.22118134e-04, 4.06074687e-04,
        1.36724850e-02, 8.39441642e-03, 8.89237039e-04, 1.82748935e-03,
        3.41706201e-02, 6.23350381e-04, 1.23085314e-03, 1.226273

In [ ]:
print(f"Label {np.argmax(y_hat[0])} predicted for query {test_text[0]}")
print(f"Label {np.argmax(y_hat[1])} predicted for query {test_text[1]}")
print(f"Label {np.argmax(y_hat[2])} predicted for query {test_text[2]}")

Label 8 predicted for query Hi! This is sairam from DLCVNLP batch.
Label 6 predicted for query Good Morning
Label 93 predicted for query sir any sujjestion to overcome this error?


6  ==> Greetings <br>
93 ==> Need Help <br>
8  ==> O

With less number of epochs, results are nice for above three samples - howvere more number of epochs has to be trained

Repeat the exercise with Albert

In [ ]:
albert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3") 
albert_encoder    = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

In [ ]:
t = ['i like swami']
albert_preprocess(t)

{'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[    2,    31,   339,    13, 16374,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     

In [ ]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Text_input')
preprocessed_text = albert_preprocess(input_text)
embed_outputs = albert_encoder(preprocessed_text)

l1 = tf.keras.layers.Dropout(0.1, name='Dropout')(embed_outputs['pooled_output'])
l2 = tf.keras.layers.Dense(151, activation='softmax', name='output')(l1)

model = tf.keras.Model(inputs=[input_text], outputs=[l2])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Text_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer_5 (KerasLayer)     {'input_word_ids':   0           ['Text_input[0][0]']             
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=2)

Epoch 1/2
83/83 [==============================] - 1590s 19s/step - loss: 3.2433 - accuracy: 0.3260 - val_loss: 2.7648 - val_accuracy: 0.3766
Epoch 2/2
83/83 [==============================] - 1572s 19s/step - loss: 2.7359 - accuracy: 0.4008 - val_loss: 2.6446 - val_accuracy: 0.4312


In [ ]:
model.save('albert_saved_model/albert_multiclass_model')

In [ ]:
y_hat = model.predict(test_text)
y_hat

array([[6.87171298e-04, 2.69184704e-03, 1.54683381e-04, 1.27634604e-03,
        3.19496542e-03, 8.02720198e-04, 5.31283468e-02, 3.99560407e-02,
        3.17867985e-03, 5.15802589e-04, 2.55493657e-03, 2.85327211e-02,
        4.70891967e-03, 4.52861626e-04, 6.71974150e-04, 1.30017439e-03,
        7.10664259e-04, 1.78872712e-03, 5.36591047e-03, 3.81897087e-03,
        7.02380130e-05, 7.71609659e-04, 1.37478439e-03, 8.51185468e-04,
        2.84494599e-04, 1.66139484e-03, 5.00120176e-03, 3.86922940e-04,
        1.48145098e-03, 7.03653274e-03, 1.11895718e-03, 4.26074816e-03,
        9.16727877e-04, 4.79616938e-05, 7.87261015e-05, 3.10776843e-04,
        1.11943856e-03, 2.63209495e-05, 1.06089958e-03, 3.75072774e-03,
        1.62323134e-03, 1.77803717e-03, 7.72281492e-05, 1.34984485e-03,
        2.33757473e-03, 8.52591766e-04, 1.34054793e-03, 2.79708486e-03,
        3.39996582e-03, 7.25295907e-03, 1.51666789e-03, 4.72040754e-03,
        1.23171546e-02, 5.12769818e-03, 2.22759461e-03, 5.237604

In [ ]:
print(f"Label {np.argmax(y_hat[0])} predicted for query {test_text[0]}")
print(f"Label {np.argmax(y_hat[1])} predicted for query {test_text[1]}")
print(f"Label {np.argmax(y_hat[2])} predicted for query {test_text[2]}")

Label 93 predicted for query Hi! This is sairam from DLCVNLP batch.
Label 76 predicted for query Good Morning
Label 93 predicted for query sir any sujjestion to overcome this error?


6  ==> Greetings <br>
93 ==> Need Help <br>
8  ==> O <br>
76 ==> Good Bye

Cannot expect more accuracy as the training was executed for 2 epochs

Repeat the exercise with Distilbert

In [ ]:
distilbert_preprocess = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2") 
distilbert_encoder    = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1")

In [ ]:
t = ['i like swami']
distilbert_preprocess(t)

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[   101,    177,  16138,    187, 105097,    102,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,   

In [ ]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Text_input')
preprocessed_text = distilbert_preprocess(input_text)
embed_outputs = distilbert_encoder(preprocessed_text)

l1 = tf.keras.layers.Dropout(0.1, name='Dropout')(embed_outputs['pooled_output'])
l2 = tf.keras.layers.Dense(151, activation='softmax', name='output')(l1)

model = tf.keras.Model(inputs=[input_text], outputs=[l2])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Text_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer_7 (KerasLayer)     {'input_mask': (Non  0           ['Text_input[0][0]']             
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                      
                                                                                                  
 keras_layer_8 (KerasLayer)     {'sequence_output':  134734080   ['keras_layer_7[0][0]',    

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=2)

Epoch 1/2
83/83 [==============================] - 779s 9s/step - loss: 3.0339 - accuracy: 0.3628 - val_loss: 2.4998 - val_accuracy: 0.4801
Epoch 2/2
83/83 [==============================] - 771s 9s/step - loss: 2.3888 - accuracy: 0.4850 - val_loss: 2.3234 - val_accuracy: 0.5028


In [ ]:
model.save('distilbert_saved_model/distilbert_multiclass_model')

In [ ]:
y_hat = model.predict(test_text)
y_hat

array([[3.12653487e-04, 1.21727716e-02, 1.91726687e-03, 5.03603194e-04,
        4.08328633e-04, 1.18100527e-03, 3.80099677e-02, 6.25723694e-03,
        1.84632212e-01, 8.16299871e-04, 1.59143959e-03, 2.62109563e-02,
        3.04833124e-03, 4.91783605e-04, 4.32152592e-04, 1.86968048e-03,
        4.99140937e-04, 7.85915938e-04, 6.97561400e-03, 3.17496969e-03,
        1.38128089e-04, 8.62318964e-04, 1.86398416e-03, 6.59088546e-04,
        5.67875104e-04, 2.33430415e-03, 3.83565668e-03, 4.72335407e-04,
        1.37372524e-03, 5.87968854e-03, 6.39120070e-03, 1.25351818e-02,
        4.06589534e-04, 6.98993099e-05, 8.91761156e-05, 7.94870721e-04,
        6.04821718e-04, 7.30697866e-05, 6.21459621e-04, 1.35888215e-02,
        1.13181281e-03, 7.19546108e-04, 1.92679465e-04, 5.46803116e-04,
        8.26509553e-04, 3.46113142e-04, 5.34459774e-04, 1.03915087e-03,
        1.42450621e-02, 2.14995304e-03, 6.21325395e-04, 1.56846654e-03,
        4.45154868e-02, 2.04957835e-03, 2.52602901e-03, 3.837572

In [ ]:
print(f"Label {np.argmax(y_hat[0])} predicted for query {test_text[0]}")
print(f"Label {np.argmax(y_hat[1])} predicted for query {test_text[1]}")
print(f"Label {np.argmax(y_hat[2])} predicted for query {test_text[2]}")

Label 93 predicted for query Hi! This is sairam from DLCVNLP batch.
Label 11 predicted for query Good Morning
Label 93 predicted for query sir any sujjestion to overcome this error?


6  ==> Greetings <br>
93 ==> Need Help <br>
8  ==> O <br>
76 ==> Good Bye <br>
11 ==> Thank you